0. Loading the dataset and Pre-processing


In [ ]:
!pip install --upgrade --no-cache-dir gdown

In [ ]:
#load training and test datasets from Kaggle

#also load csv of preprocessed datasets (to reduce runtime by preprocessing once and storing the preprocessed dataset into csv)

! gdown --id 1E8OTtNK_UNSAdmkg3OuB0k2FsKKF9QpY
! gdown --id 1-pm6_kvT9CBrYER40uIkRiG0KMlHnH7L
! gdown --id 1oiVQignVg1wvE1LXKFvOLHPl0P8GNSDp

! gdown --id 1pGkEyDelD7Lp7yhFML9Dt1oo-91Mqf9H
! gdown --id 1DYVzeYjjTTeHtazHfKD9FTx57h-77bkC

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1E8OTtNK_UNSAdmkg3OuB0k2FsKKF9QpY
To: /content/train.csv
100% 98.6M/98.6M [00:01<00:00, 51.5MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-pm6_kvT9CBrYER40uIkRiG0KMlHnH7L
To: /content/test.csv
100% 25.1M/25.1M [00:01<00:00, 24.3MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
submit_data = pd.read_csv("submit.csv")

In [ ]:
test_data = test_data.join(submit_data["label"])

In [ ]:
#preprocess train data

In [ ]:
# Checking null values
train_data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
# Removing null values
train_data.dropna(inplace=True)
train_data.drop(train_data.loc[train_data['text']==" "].index, inplace=True)
train_data.drop(train_data.loc[train_data['text']=="  "].index, inplace=True)
train_data.drop(train_data.loc[train_data['text']=="\n"].index, inplace=True)

# Resetting index
train_data.reset_index(inplace=True)

In [ ]:
train_data= train_data.apply(lambda x: x.astype(str).str.lower())

In [ ]:
train_data = train_data.drop(columns=["id", "index"])

In [ ]:
#preprocess test data

In [ ]:
# Checking null values
test_data.isnull().sum()

id          0
title     122
author    503
text        7
label       0
dtype: int64

In [ ]:
# Removing null values
test_data.dropna(inplace=True)
test_data.drop(test_data.loc[test_data['text']==" "].index, inplace=True)
test_data.drop(test_data.loc[test_data['text']=="  "].index, inplace=True)
test_data.drop(test_data.loc[test_data['text']=="\n"].index, inplace=True)

# Resetting index
test_data.reset_index(inplace=True)

In [ ]:
test_data= test_data.apply(lambda x: x.astype(str).str.lower())

In [ ]:
test_data = test_data.drop(columns=["id","index"])

In [ ]:
# train_data.shape

In [ ]:
transformed_df = pd.read_csv("pre_train.csv")
transformed_df_test = pd.read_csv("pre_test.csv")

In [ ]:
X_test = transformed_df_test.values
y_test = test_data["label"].values 

In [ ]:
X = transformed_df.values
y = train_data["label"].values

In [ ]:
#Kfold (k=5) split for training data

from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5)

X_train=[]
X_val=[]
y_train=[]
y_val=[]

for train_index, test_index in skf.split(X, y):
  X_train.append(X[train_index])
  X_val.append(X[test_index]) 
  y_train.append(y[train_index])
  y_val.append(y[test_index])


1. Hyperparameter Tuning using Bayesian Optimization Search

In [ ]:
! pip install scikit-optimize

     |████████████████████████████████| 100 kB 7.9 MB/s 


In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.linear_model import SGDClassifier

1.1 Hyperparameter Tuning for SVM(SGD)

In [ ]:
model_svm_sgd = SGDClassifier(loss='hinge', random_state=0)

In [ ]:

search_space = {"alpha": Real(0, 1), 
                "eta0" : Real(0.1,1),
        "penalty": Categorical(['l1', 'l2', 'elasticnet']),
        "early_stopping": Categorical([True, False]),
        "learning_rate": Categorical(['constant', 'invscaling']),
        "max_iter": Integer(1000, 3000),
    
    }

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=100)
pca_X = pca.fit_transform(X)

In [ ]:
SVM_sgd_bayes_search = BayesSearchCV(model_svm_sgd, search_space, n_iter=32, # specify how many iterations
                                    n_jobs=-1, cv=5)

In [ ]:
SVM_sgd_bayes_search.fit(pca_X, y)

/usr/local/lib/python3.7/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


BayesSearchCV(cv=5, estimator=SGDClassifier(random_state=0), n_iter=32,
              n_jobs=-1,
              search_spaces={'alpha': Real(low=0, high=1, prior='uniform', transform='normalize'),
                             'early_stopping': Categorical(categories=(True, False), prior=None),
                             'eta0': Real(low=0.1, high=1, prior='uniform', transform='normalize'),
                             'learning_rate': Categorical(categories=('constant', 'invscaling'), prior=None),
                             'max_iter': Integer(low=1000, high=3000, prior='uniform', transform='normalize'),
                             'penalty': Categorical(categories=('l1', 'l2', 'elasticnet'), prior=None)})

In [ ]:
SVM_sgd_best_params = SVM_sgd_bayes_search.best_params_
SVM_sgd_best_params

OrderedDict([('alpha', 0.0),
             ('early_stopping', False),
             ('eta0', 0.1),
             ('learning_rate', 'constant'),
             ('max_iter', 3000),
             ('penalty', 'l2')])

In [ ]:
print(SVM_sgd_bayes_search.best_score_)

0.9233429701941678


1.2 Hyperparameter Tuning for SVM (SMO)

In [ ]:
#Hyperparam tuning for SVM

from sklearn.svm import SVC
model_svm = SVC(random_state=0)

search_space = {"C": Real(1, 3), 
        "kernel": Categorical(['linear', 'poly', 'rbf', 'sigmoid']),
        "shrinking": Categorical([True, False]), 
        "max_iter": Integer(1000, 3000),
        "tol": Real(1e-4,1e-3)
    
    }

In [ ]:
SVM_bayes_search = BayesSearchCV(model_svm, search_space, n_iter=32, # specify how many iterations
                                    n_jobs=-1, cv=5)

In [ ]:
SVM_bayes_search.fit(pca_X, y)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


BayesSearchCV(cv=5, estimator=SVC(random_state=0), n_iter=32, n_jobs=-1,
              search_spaces={'C': Real(low=1, high=3, prior='uniform', transform='normalize'),
                             'kernel': Categorical(categories=('linear', 'poly', 'rbf', 'sigmoid'), prior=None),
                             'max_iter': Integer(low=1000, high=3000, prior='uniform', transform='normalize'),
                             'shrinking': Categorical(categories=(True, False), prior=None),
                             'tol': Real(low=0.0001, high=0.001, prior='uniform', transform='normalize')})

In [ ]:
SVM_best_params = SVM_bayes_search.best_params_
SVM_best_params

OrderedDict([('C', 3.0),
             ('kernel', 'rbf'),
             ('max_iter', 3000),
             ('shrinking', False),
             ('tol', 0.0008827394667177391)])

In [ ]:
SVM_best_model = SVM_bayes_search.best_estimator_
SVM_best_model

SVC(C=3.0, max_iter=3000, random_state=0, shrinking=False,
    tol=0.0008827394667177391)

In [ ]:
print(SVM_bayes_search.best_score_)

0.9453626017745511


1.3 Hyperparameter Tuning for DT 

In [ ]:
#Hyperparam tuning for DT

from sklearn.tree import DecisionTreeClassifier
model_DT = DecisionTreeClassifier(random_state=0)

search_space = {"criterion": Categorical(['gini', 'entropy']), 
        "max_depth": Integer(3,15),
        "min_samples_split": Integer(2,10), 
        "min_samples_leaf": Integer(1,5),
        "ccp_alpha": Real(0, 2)
    }

In [ ]:
DT_bayes_search = BayesSearchCV(model_DT, search_space, n_iter=50, # specify how many iterations
                                    n_jobs=-1, cv=5)

In [ ]:
DT_bayes_search.fit(pca_X, y)

In [ ]:
DT_best_params = DT_bayes_search.best_params_
DT_best_params

In [ ]:
DT_best_model = DT_bayes_search.best_estimator_
DT_best_model

In [ ]:
print(DT_bayes_search.best_score_)

1.4 Hyperparameter Tuning for MLP (Quasi-Newton)

In [ ]:
#Hyperparam tuning for MLP (Quasi-Newton)

from sklearn.neural_network import MLPClassifier
model_n_MLP = MLPClassifier(solver='lbfgs', random_state=0)

search_space = {"activation": Categorical(['identity', 'logistic', 'tanh', 'relu']), 
        "alpha": Real(0,2), 
        "learning_rate": Categorical(['constant', 'invscaling', 'adaptive']),
        "early_stopping": Categorical([True, False]),
        "max_iter": Integer(1000, 3000)
    }

In [ ]:
MLP_n_bayes_search = BayesSearchCV(model_n_MLP, search_space, n_iter=32, # specify how many iterations
                                    n_jobs=-1, cv=5)

In [ ]:
MLP_n_bayes_search.fit(pca_X, y)

In [ ]:
MLP_n_best_params = MLP_n_bayes_search.best_params_
MLP_n_best_params

In [ ]:
print(MLP_n_bayes_search.best_score_)

1.5 Hyperparameter Tuning for MLP (SGD) 

In [ ]:
from sklearn.neural_network import MLPClassifier
model_MLP = MLPClassifier(random_state=0)

search_space = {"activation": Categorical(['identity', 'logistic', 'tanh', 'relu']), 
        "solver": Categorical(['lbfgs', 'sgd', 'adam']),
        "alpha": Real(0,2), 
        "learning_rate": Categorical(['constant', 'invscaling', 'adaptive']),
        "early_stopping": Categorical([True, False]),
        "max_iter": Integer(1000, 3000)
    }

In [ ]:
MLP_bayes_search = BayesSearchCV(model_MLP, search_space, n_iter=50, # specify how many iterations
                                    n_jobs=-1, cv=5)

In [ ]:
MLP_bayes_search.fit(pca_X, y)

In [ ]:
MLP_best_params = MLP_bayes_search.best_params_
MLP_best_params

In [ ]:
MLP_best_model = MLP_bayes_search.best_estimator_
MLP_best_model

In [ ]:
print(MLP_bayes_search.best_score_)